- Change the np.load to include the file
- Add the following .pkl files into the same folder as this .ipynb (model, model_features, scaler, poly)
- Edit the transform_features() method if necessary

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import entropy
import joblib
import networkx as nx
import sys
import os

sys.path.append(os.path.abspath(".."))

test=np.load("../datasets/labeled/third_batch_multi_labels.npz")

X_test=test["X"]

XX_test = pd.DataFrame(X_test)
XX_test.rename(columns={0:"user",1:"item",2:"rating"},inplace=True)

num_unique_users = XX_test["user"].nunique()
print(f"Number of unique user IDs in the test set: {num_unique_users}")

Number of unique user IDs in the test set: 1100


In [2]:
from utils.feature_transformation import aggregate_features
from sklearn.impute import SimpleImputer

test_features = aggregate_features(XX_test)
test_features.sort_values(by="user", inplace=True)

# Select only important features
model_features = joblib.load("model_features.pkl")
print(f"test_features before selecting features {test_features.shape}")

# If feature does not exist, populate with 0s
for feat in model_features:
    if feat not in test_features.columns:
        test_features[feat] = 0
test_features = test_features[model_features]
print(f"test_features after selecting features {test_features.shape}")

# debugging
# print(test_features.columns)

scaler = joblib.load("scaler.pkl")
poly = joblib.load("poly.pkl")

# Impute missing values (e.g., mean or median strategy)
# imputer = SimpleImputer(strategy='median')
# test_features_imputed = imputer.fit_transform(test_features)

test_features_scaled = scaler.transform(test_features)
test_features_poly = poly.transform(test_features_scaled)

# Load the trained model and predict probabilities (shape: #test_users x 6)
xgb_model = joblib.load("model.pkl")
probabilities = xgb_model.predict_proba(test_features_poly)
y_pred_proba_rf = xgb_model.predict_proba(test_features_poly)
# print(y_pred_proba_rf)

np.savez("predictions.npz", probabilities=probabilities)
print(f"prediction shape {probabilities.shape}")

test_results=np.load("predictions.npz")
test_results_df = pd.DataFrame(test_results["probabilities"])
test_results_df.head()

test_features before selecting features (1100, 57)
test_features after selecting features (1100, 41)
prediction shape (1100, 6)


,0,1,2,3,4,5
0,0.992915,0.000479,0.002224,0.000039,0.000996,0.003348
1,0.991937,0.001922,0.000742,0.000059,0.004433,0.000908
2,0.929539,0.011728,0.000631,0.000395,0.055746,0.001961
3,0.996356,0.001894,0.000132,0.000133,0.001200,0.000285
4,0.993616,0.000992,0.000110,0.000035,0.000678,0.004569


In [3]:
data = np.load('predictions.npz')
predictions = data['probabilities']

class_counts = {i: 0 for i in range(6)}

for row in predictions:
    predicted_class = np.argmax(row)
    class_counts[predicted_class] += 1

print("Class instance counts:")
for class_label, count in class_counts.items():
    print(f"Class {class_label}: {count}") 

Class instance counts:
Class 0: 1003
Class 1: 7
Class 2: 15
Class 3: 27
Class 4: 19
Class 5: 29


In [4]:
from sklearn.metrics import roc_auc_score

y_true = test["y"]

# Convert true labels to a DataFrame
df_y_true = pd.DataFrame(y_true, columns=["user", "true_label"])

# Load the predicted probabilities
predictions_data = np.load("predictions.npz")
probabilities = predictions_data["probabilities"]

predicted_labels = np.argmax(probabilities, axis=1)

df_predictions = pd.DataFrame({
    "user": df_y_true["user"],
    "true_label": df_y_true["true_label"],
    "predicted_label": predicted_labels
})

# Identify misclassified users
df_predictions["correct"] = df_predictions["true_label"] == df_predictions["predicted_label"]
df_misclassified = df_predictions[df_predictions["correct"] == False]
# df_misclassified.head(2)
# df_misclassified.to_csv("misclassified_users.csv", index=False)

auc_per_class = {}
for i in range(probabilities.shape[1]):
    binary_true = (df_predictions["true_label"] == i).astype(int)
    try:
        auc = roc_auc_score(binary_true, probabilities[:, i])
        auc_per_class[i] = auc
        print(f"  Class {i}: AUC = {auc:.3f}")
    except ValueError:
        auc_per_class[i] = None

k = 5
AUC_0 = auc_per_class[0]
anomaly_aucs = [auc_per_class[i] for i in range(1, k+1) if i in auc_per_class]

final_metric = (0.5 * AUC_0) + (0.5 / k) * sum(anomaly_aucs)
print(f"\n🏆 Final Evaluation Metric: {final_metric:.3f}")

# Convert AUC scores to DataFrame
df_auc = pd.DataFrame(list(auc_per_class.items()), columns=["class", "AUC"])


  Class 0: AUC = 0.899
  Class 1: AUC = 0.854
  Class 2: AUC = 0.857
  Class 3: AUC = 0.986
  Class 4: AUC = 0.820
  Class 5: AUC = 0.754

🏆 Final Evaluation Metric: 0.876
